In [60]:
import numpy as np
np.random.seed(42)
#step 1: initialize PD for borrowers, sampling from uniform dist
borrower=np.random.uniform(0,0.05,(1,1000))
borrower



array([[0.01872701, 0.04753572, 0.0365997 , 0.02993292, 0.00780093,
        0.00779973, 0.00290418, 0.04330881, 0.03005575, 0.03540363,
        0.00102922, 0.04849549, 0.04162213, 0.01061696, 0.00909125,
        0.00917023, 0.01521211, 0.02623782, 0.02159725, 0.01456146,
        0.03059264, 0.00697469, 0.01460723, 0.01831809, 0.0228035 ,
        0.0392588 , 0.00998369, 0.02571172, 0.02962073, 0.00232252,
        0.03037724, 0.00852621, 0.00325258, 0.04744428, 0.0482816 ,
        0.04041987, 0.01523069, 0.00488361, 0.03421165, 0.02200762,
        0.00610191, 0.02475885, 0.00171943, 0.04546602, 0.012939  ,
        0.03312611, 0.01558555, 0.0260034 , 0.02733551, 0.00924272,
        0.04847923, 0.03875664, 0.04697495, 0.04474137, 0.029895  ,
        0.04609371, 0.00442463, 0.00979914, 0.00226136, 0.01626652,
        0.01943386, 0.01356745, 0.04143688, 0.01783767, 0.01404673,
        0.0271348 , 0.00704621, 0.04010985, 0.00372753, 0.04934435,
        0.03861224, 0.00993578, 0.00027611, 0.04

In [49]:
#step 2: simulate a five-year history based on unemployment rate
# UR based on AR model: UR_t = 0.01 + 0.8 UR_t-1 + epsilon_t-1

UR=np.zeros(20)
epsilon=np.random.normal(0,0.0025,20)

# UR at time zero will be assumed to be 3.6%
UR[0]=0.036
for i in range(19):
    UR[i+1]=0.01+0.8 * UR[i]+epsilon[i]

#We will assume that, no matter what, the UR at the end of the 5th simulation quarter and the 13th simulation quarter is 10%.
UR[5]=UR[13]=0.1

UR





array([0.036     , 0.03803493, 0.03459726, 0.03695885, 0.04196691,
       0.1       , 0.03883789, 0.03846373, 0.04150187, 0.04171017,
       0.0445491 , 0.04996957, 0.04988321, 0.1       , 0.05357076,
       0.05560983, 0.05134236, 0.05163135, 0.0463249 , 0.04370894])

In [50]:
#step 3: create PD multiple by centering UR 
PD_multiple = UR/UR.mean()
PD_multiple

array([0.72386425, 0.76478129, 0.69565896, 0.74314414, 0.84384299,
       2.01073402, 0.78092664, 0.77340327, 0.83449215, 0.83868055,
       0.89576392, 1.00475517, 1.00301866, 2.01073402, 1.07716555,
       1.11816581, 1.03235828, 1.0381692 , 0.93147053, 0.87887062])

In [51]:
#step 4: #Convert each default rate into a quarterly
borrower_quarterly_basis = 1 - (1 - borrower )**(1/4)


In [ ]:
#step 5: simulate the PD for each borrower at each each quarter by multipler

borrower_quarterly=np.zeros((1000,20))

for i in range(20):
    borrower_quarterly[:,i] = borrower_quarterly_basis * PD_multiple [i]


print(borrower_quarterly.mean(),borrower_quarterly.min(),borrower_quarterly.max())





0.006354377015113268 1.707629508172768e-05 0.025607948068121417


In [75]:
#step 6: Simulate the default history, by rolling one uniform random 
# for each surviving customer and seeing if it is less than the quarterly PD 
# If it is, mark that customer as defaulted.

#default = np.zeros((1,1000))

#for t in range(20):
    #for j in range(1000):
        #if np.random.uniform() < borrower_quarterly[j,t]:
            #default[0,j]=1

#default

default = np.zeros((1000,20))
alive=np.full(21,1000)

for t in range(20):
    for j in range(1000):
        if np.random.uniform() < borrower_quarterly[j,t]:
            default[j,t]=1
            alive[t+1]=alive[t]-default[:,t].sum()

default.sum()
alive




array([1000,  996,  994,  985,  981,  973,  960,  956,  949,  946,  939,
        931,  926,  919,  909,  904,  893,  889,  885,  877,  871])

In [ ]:
#step 7: default rate 
#default_rate = default.sum()/1000
#default_rate

quarter_pooled_default= default.sum()/alive[:-1].sum()
annual_default=1-(1-quarter_pooled_default)**4

quarter_pooled_default


0.006857325111630874